# Facial Expression Recognition

The project has the following stages:
1. Collecting images with expressions
2. Detecting faces in the images and saving them
3. Training our classifier on the Faces
4. Recognition of expression in new images

> Let's first clone the code from our GitHub public repository

In [1]:
## !git clone https://github.com/codesapienbe/facial_expressions.git

import os

import git
from git import RemoteProgress

git_root = "pure_face_git"
git_repo = "https://github.com/codesapienbe/facial_expressions.git"


class CloneProgress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        if message:
            # clean up the message a bit
            message = message.strip()
            if message.endswith('.'):
                message = message[:-1]
            if message.startswith('remote: '):
                message = message[8:]

            # clear the current line and print the message
            print('\r', end='')
            print(message)

print('Cloning into %s' % git_root)
git.Repo.clone_from(git_repo, git_root, progress=CloneProgress())

# change the current working directory to the cloned repository
os.chdir("pure_face_git")

Cloning into pure_face_git
2.04 MiB | 4.07 MiB/s
4.32 MiB | 4.32 MiB/s
6.43 MiB | 4.27 MiB/s
7.93 MiB | 3.95 MiB/s
7.93 MiB | 3.95 MiB/s
10.89 MiB | 3.62 MiB/s
11.39 MiB | 3.24 MiB/s
12.09 MiB | 3.01 MiB/s
14.07 MiB | 2.66 MiB/s
14.07 MiB | 2.66 MiB/s
16.94 MiB | 2.33 MiB/s
16.94 MiB | 2.33 MiB/s
18.62 MiB | 2.37 MiB/s
19.51 MiB | 2.18 MiB/s
20.54 MiB | 2.14 MiB/s
21.77 MiB | 2.30 MiB/s
23.20 MiB | 2.46 MiB/s
24.87 MiB | 2.64 MiB/s
26.75 MiB | 2.81 MiB/s
28.80 MiB | 2.97 MiB/s
28.80 MiB | 2.97 MiB/s
31.07 MiB | 3.13 MiB/s
32.44 MiB | 3.06 MiB/s
33.79 MiB | 3.17 MiB/s
35.34 MiB | 3.28 MiB/s
37.10 MiB | 3.40 MiB/s
39.04 MiB | 3.51 MiB/s
39.04 MiB | 3.51 MiB/s
41.21 MiB | 3.62 MiB/s
43.57 MiB | 3.73 MiB/s
43.57 MiB | 3.73 MiB/s
46.13 MiB | 3.84 MiB/s
48.25 MiB | 3.80 MiB/s
48.25 MiB | 3.80 MiB/s
50.40 MiB | 3.98 MiB/s
52.76 MiB | 4.20 MiB/s
52.76 MiB | 4.20 MiB/s
55.21 MiB | 4.41 MiB/s
57.09 MiB | 4.43 MiB/s
57.09 MiB | 4.43 MiB/s
57.09 MiB | 4.43 MiB/s
59.15 MiB | 4.46 MiB/s
61.38 MiB | 

Let us create required directory structure

In [42]:
# if the directory does not exist, create it
if not os.path.exists("data_set"):
    os.mkdir("data_set")

if not os.path.exists("data_set/anger"):
    os.mkdir("data_set/anger")

if not os.path.exists("data_set/happy"):
    os.mkdir("data_set/happy")

if not os.path.exists("data_set/neutral"):
    os.mkdir("data_set/neutral")

if not os.path.exists("data_set/sad"):
    os.mkdir("data_set/sad")

if not os.path.exists("data_set/surprise"):
    os.mkdir("data_set/surprise")

Extracting Images with Expressions

In [43]:
import cv2
import numpy as np
from PIL import Image

In [44]:
def generate_emotion_data_set(emotion_title):
    global f, img, image, source, destination
    with open(f"{emotion_title}.txt", 'r') as f:
        img = [line.strip() for line in f]
    # loading images
    for image in img:
        source = cv2.imread("images/" + image)
        destination = f"data_set/{emotion_title}/" + image
        cv2.imwrite(destination, source)  # 250 images


In [45]:
generate_emotion_data_set("anger")
generate_emotion_data_set("happy")
generate_emotion_data_set("neutral")
generate_emotion_data_set("sad")
generate_emotion_data_set("surprise")

Create a new folder for the data set called 'ds'

In [46]:
os.mkdir("ds")

## Creating Data Set of Faces

In [47]:
def generate_dataset_of_faces(fid, emotion_title):
    global f, images, face_detector, face_id, count, image, img, gray, faces, x, y, w, h
    with open(f"{emotion_title}.txt", 'r') as f:
        images = [line.strip() for line in f]
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    face_id = fid
    count = 0
    for image in images:
        img = cv2.imread(f"data_set/{emotion_title}/" + image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            count += 1

            # Save the captured image into the datasets folder
            cv2.imwrite("ds/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y + h, x:x + w])


In [48]:
generate_dataset_of_faces(0, "anger")
generate_dataset_of_faces(1, "happy")
generate_dataset_of_faces(2, "neutral")
generate_dataset_of_faces(3, "sad")
generate_dataset_of_faces(4, "surprise")

display("\nDatasets are created successfully")

'\nDatasets are created successfully'

Let us make a new folder for the training data

In [49]:
os.mkdir("trainer")

In [50]:
# Path for face image database
path = 'ds'

recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");


# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []

    for imagePath in imagePaths:

        PIL_img = Image.open(imagePath).convert('L')  # convert it to grayscale
        img_numpy = np.array(PIL_img, 'uint8')

        # id is the name of the image file
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        # faces are the faces in the image
        faces = detector.detectMultiScale(img_numpy)

        # x,y,w,h are the coordinates of the face in the image
        for (x, y, w, h) in faces:
            faceSamples.append(img_numpy[y:y + h, x:x + w])
            ids.append(id)

    return faceSamples, ids

In [51]:
print("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces, ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml')  # recognizer.save() worked on Mac, but not on Pi

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 5 faces trained. Exiting Program


## Recognizing Faces (Testing)

In [52]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

# initiate id counter
id = 0

# names related to ids: example ==> Marcelo: id=1,  etc
names = ['Anger', 'Happy', 'Neutral', 'Sad', 'Surprise']

# ask user to select a video file
selection = input("Select a video file to test the model (Default: Webcam (0) or enter the absolute path to a video file):")

if selection == "0" or selection == "" or selection is None:
    cam = cv2.VideoCapture(0)
else:
    cam = cv2.VideoCapture(selection)

# set the resolution of the camera to full screen size
# get the width and height of the screen
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
# set the width and height, and UNSUCCESSFULLY set the exposure time
cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cam.set(cv2.CAP_PROP_FPS, 60)

# Define min window size to be recognized as a face
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:

    ret, img = cam.read()
    img = cv2.flip(img, 1)  # Flip vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    for (x, y, w, h) in faces:

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

        id, confidence = recognizer.predict(gray[y:y + h, x:x + w])

        # Check if confidence is less them 100 ==> "0" is perfect match
        if confidence < 100:
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))

        cv2.putText(img, str(id), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, str(confidence), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)

    cv2.imshow('camera', img)

    k = cv2.waitKey(10) & 0xff  # Press 'ESC' for exiting video
    if k == 27:
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program and cleanup stuff
